In [15]:
import pandas as pd
import numpy as np
from binance.client import Client
import yfinance as yf
import pickle
import requests
from requests.auth import HTTPBasicAuth
import hashlib
from urllib.parse import urlencode
import hmac

%run helper_functions.ipynb
%run train.ipynb

Testing set size: 429
Model guessed up or down movement correctly 0.6107226107226107%
mae: 0.02008951424594497
mse: 0.0009766826401910757
R^2: 0.3159961409202995


# Get data
Get latest data and features to predict next days price

In [9]:
hist = get_yfinance_data('BTC-USD')[['Close']].rename(columns = {'Close': 'Btc Close'}) # get btc hist
hist['Btc Returns'] = hist['Btc Close'].pct_change() # get btc returns

# Lagged Moving average - Btc price
windows = [3, 5, 7, 10, 20, 50, 100, 200] # set moving average windows
for window in windows:
    hist['Btc Close - ' +str(window) + 
         'd Avg'] = hist['Btc Close'] - moving_average(hist['Btc Close'], window=window, lag=0)

# Predict and Trade
Connect to API <br>
Make trade based on prediction. <br>
Scenario 1: Positive return pred. If in bitcoin, stay. Else out of bitcoin, buy. <br>
Scenario 2: Negative return pred. If in bitcoin, sell. Else out of bitcoin, stay. <br>

### Load Model and predict

In [10]:
loaded_model = pickle.load(open('model_weights.sav', 'rb')) # load model
pred = loaded_model.predict(hist[hist.columns[2:]].tail(1)) # make prediction on latest data

In [12]:
pred

array([0.026093])

### Connect API and get balance

In [13]:
api_key = {'X-MBX-APIKEY' : 'UELhfQrZqnrZnYUJt8916bK4CMCQfv8dCd7lPFYaZiJe0jTuHT3LuZlJGwQIrpza'} # Set API key
sec_key = 'Id0twv9OWI1v4VlgO6abbXt0t45TWGbYaDUxgRRSopk2eLMKTysl5vPaktvtyJVt' # Set API secret
server_time = get_binance_time() # get binance server time

params = binance_params(secret = sec_key, timestamp=server_time, recvWindow=6000)
balances = requests.get('https://api.binance.com/api/v3/account', headers=api_key, params=params).json()   

btc_balance = float([balance for balance in balances['balances'] if balance['asset'] == 'BTC'][0]['free'])

### Make Trade

In [ ]:
if pred > 0:
    if btc_balance == 0:
        # buy

if pred < 0:
    if btc_balance > 0:
        # sell